# How to select examples from a LangSmith dataset

import Prerequisites from "@theme/Prerequisites";
import Compatibility from "@theme/Compatibility";

<Prerequisites titlesAndLinks={[
  ["Chat models", "/docs/concepts/#chat-models"],
  ["Few-shot-prompting", "/docs/concepts/#few-shot-prompting"],
  ["LangSmith", "/docs/concepts/#langsmith"],
]} />


<Compatibility packagesAndVersions={[
  ["langsmith", "0.1.101"],
]} />


LangSmith datasets have built-in support for similarity search, making them a great tool for building and querying few-shot examples.

In this guide we'll see how to use an indexed LangSmith dataset as a few-shot example selector.

## Setup

Before getting started make sure you've [created a LangSmith account](https://smith.langchain.com/) and set your credentials:

In [1]:
import getpass
import os

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Set LangSmith API key:\n\n")

os.environ["LANGSMITH_TRACING"] = "true"

Set LangSmith API key:

········


We'll need to install the `langsmith` SDK. In this example we'll also make use of `langchain`, `langchain-openai`, and `langchain-benchmarks`:

In [ ]:
%pip install -qU langsmith>=0.1.101 langchain langchain-openai langchain-benchmarks

Now we'll clone a public dataset and turn on indexing for the dataset. We can also turn on indexing via the [LangSmith UI](https://docs.smith.langchain.com/how_to_guides/datasets/index_datasets_for_dynamic_few_shot_example_selection).

We'll clone the [Multiverse math few shot example dataset](https://blog.langchain.dev/few-shot-prompting-to-improve-tool-calling-performance/).

This enables searching over the dataset, and will make sure that anytime we update/add examples they are also indexed.

In [ ]:
from langsmith import AsyncClient as AsyncLangSmith
from langsmith import Client as LangSmith

ls_client = LangSmith()
async_ls_client = AsyncLangSmith()

dataset_name = "multiverse-math-examples-for-few-shot"
dataset_public_url = (
    "https://smith.langchain.com/public/0df59e49-d226-4ef2-9ecd-8c0fc9cd0288/d"
)

ls_client.clone_public_dataset(dataset_public_url)

dataset_id = ls_client.read_dataset(dataset_name=dataset_name).id
ls_client.index_dataset(dataset_id=dataset_id)

Indexing can take a few seconds. Once the dataset is indexed, we can search for similar examples like so:

In [29]:
examples = ls_client.similar_examples(
    {"input": "whats the negation of the negation of the negation of 3"},
    limit=3,
    dataset_id=dataset_id,
)
len(examples)

3

In [34]:
examples[0].inputs["input"]

'evaluate the negation of -100'

For this dataset the outputs are an entire chat history:

In [33]:
len(examples[1].outputs["output"])

9

The search returns the examples whose inputs are most similar to the query input. We can use this for few-shot prompting a model like so:

In [50]:
from langchain.chat_models import init_chat_model
from langchain_benchmarks.tool_usage.tasks.multiverse_math import (
    add,
    cos,
    divide,
    log,
    multiply,
    negate,
    pi,
    power,
    sin,
    subtract,
)
from langchain_core.messages import HumanMessage, SystemMessage, convert_to_messages
from langchain_core.runnables import RunnableLambda


def similar_examples(input_: dict) -> dict:
    examples = ls_client.similar_examples(input_, limit=5, dataset_id=dataset_id)
    return {**input_, "examples": examples}


async def asimilar_examples(input_: dict) -> dict:
    examples = await async_ls_client.similar_examples(
        input_, limit=5, dataset_id=dataset_id
    )
    return {**input_, "examples": examples}


def construct_prompt(input_: dict) -> list:
    instructions = """You are great at using mathematical tools."""
    messages = []
    for ex in input_["examples"]:
        # For this dataset, a multi-turn conversation is stored as output.
        messages.extend(convert_to_messages(ex.outputs["output"]))
    examples = [msg for msg in messages if not isinstance(msg, SystemMessage)]
    for ex in examples:
        ex.name = (
            "example_user" if isinstance(ex, HumanMessage) else "example_assistant"
        )
    return [SystemMessage(instructions), *examples, HumanMessage(input_["input"])]


tools = [add, cos, divide, log, multiply, negate, pi, power, sin, subtract]
llm = init_chat_model("gpt-4o")
llm_with_tools = llm.bind_tools(tools)

example_selector = RunnableLambda(func=similar_examples, afunc=asimilar_examples)

chain = example_selector | construct_prompt | llm_with_tools

In [52]:
ai_msg = await chain.ainvoke({"input": "whats the negation of the negation of 3"})
ai_msg.tool_calls

[{'name': 'negate',
  'args': {'a': 3},
  'id': 'call_ehmx3Z4Cj6HFpI8FV4pYZ5Oo',
  'type': 'tool_call'}]

Looking at the LangSmith trace, we can see that relevant examples were pulled in in the `similar_examples` step and passed as messages to ChatOpenAI: https://smith.langchain.com/public/05af2ce8-1a45-4f3a-8d54-6524ff919279/r.